In [6]:
import { load } from "dotenv";
const env = await load({
  envPath:'./.env.local',
  export: true
});

const process = {
    env
}

const chatOptions = {
  openAIApiKey: process.env.OPENAI_API_KEY,
  temperature: 1.5,
  model: "deepseek-chat",
  configuration: {
    baseURL: "https://api.deepseek.com",
  },
  azureOpenAIBasePath: "https://api.deepseek.com",
}

console.log(process.env)

{
  LANGSMITH_TRACING: "true",
  LANGSMITH_ENDPOINT: "https://api.smith.langchain.com",
  LANGSMITH_API_KEY: "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",
  LANGSMITH_PROJECT: "chat-next",
  OPENAI_API_KEY: "sk-4714954f89824d7aa87c44c2b22c706e",
  LANGCHAIN_VERBOSE: "true",
  LANGCHAIN_DEBUG: "true",
  NEXT_PUBLIC_USE_MOCK: "true"
}


langchain 的核心接口 `Runnable`，是一个流式接口，接口上有几个常用方法
- `invoke` 基础调用
- `batch` 批量调用
- `stream` 调用并以流式返回
- `streamLog` 将多个 `Runnable` 串联起来
- `pipe` 将多个 `Runnable` 串联起来


In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatModel = new ChatOpenAI(chatOptions);

await chatModel.invoke([
    new HumanMessage("Tell me a joke")
])

const outputPrase = new StringOutputParser()

const simpleChain = chatModel.pipe(outputPrase)

await simpleChain.invoke([new HumanMessage("Tell me a joke")])


批量调用

In [ ]:
await simpleChain.batch([
  [ new HumanMessage("Tell me a joke") ],
  [ new HumanMessage("Hi, Who are you?") ],
])

流式输出

In [4]:
const stream = await simpleChain.stream([
  new HumanMessage("Tell me a joke")
])

for await (const chunk of stream){
 console.log(chunk)
}

 you
:


Why
 don
’
t
 skeletons
 fight
 each
 other
?
  

Because
 they
 don
’
t
 have
 the
 guts
!
 😄


[llm/end] [1:chain:RunnableSequence > 2:llm:ChatOpenAI] [1.80s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "Sure, here's a joke for you:\n\nWhy don’t skeletons fight each other?  \nBecause they don’t have the guts! 😄",
        "generationInfo": {
          "prompt": 0,
          "completion": 0,
          "finish_reason": "stop",
          "system_fingerprint": "fp_3a5770e1b4",
          "model_name": "deepseek-chat"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "AIMessageChunk"
          ],
          "kwargs": {
            "content": "Sure, here's a joke for you:\n\nWhy don’t skeletons fight each other?  \nBecause they don’t have the guts! 😄",
            "additional_kwargs": {},
            "response_metadata": {
         

## fallback

回退机制

任何一个 Runnable 都可以设置回退机制，当调用失败时，会调用回退机制

In [8]:
import { ChatOpenAI } from "@langchain/openai"

// 模拟一个失败的情况
const fakeLLM = new ChatOpenAI({
  azureOpenAIApiKey: "123",
  maxRetries: 0,
})

// 回退到另一个模型
const realLLM = new ChatOpenAI(chatOptions)
// 设置回退机制
const llmWithFallback = fakeLLM.withFallbacks({
  fallbacks: [realLLM],
})

await llmWithFallback.invoke("你好")


[chain/start] [1:chain:RunnableWithFallbacks] Entering Chain run with input: {
  "input": "你好"
}
[llm/start] [1:chain:RunnableWithFallbacks > 2:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "你好",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "你好",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/error] [1:chain:RunnableWithFallbacks > 2:llm:ChatOpenAI] s] LLM run errored with error: "

AIMessage {
  "id": "9d6a7b02-10fd-40a6-992e-7977c983e785",
  "content": "你好！有什么我可以帮忙的吗？",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 4,
      "completionTokens": 7,
      "totalTokens": 11
    },
    "finish_reason": "stop",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 4,
      "completion_tokens": 7,
      "total_tokens": 11,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "prompt_cache_hit_tokens": 0,
      "prompt_cache_miss_tokens": 4
    },
    "system_fingerprint": "fp_3a5770e1b4"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 7,
    "input_tokens": 4,
    "total_tokens": 11,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_details": {}
  }
}